In [ ]:
from utils import *
from tst import Transformer


In [ ]:
class SimpleModel(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        self.transformer = Transformer(d_input=220, d_model=1024, d_output=16, q=8, v=8, h=20, N=20, chunk_mode=None)
        self.target_year_linear = nn.Linear(9, 16)
        self.intermediate_linear = nn.Linear(192, 32)
        self.prediction_head = nn.Linear(32, 1)
        self.activation = nn.ReLU()
        self.batchnorm = nn.BatchNorm1d(32)
    
    def forward(self, X, x):
        
        transformer_features = self.transformer(X)
        
        x_features = self.target_year_linear(x)
                
        transformer_features = transformer_features.reshape(transformer_features.shape[0], -1)
        
        joint_features = torch.cat([transformer_features, x_features], dim=-1)
        
        joint_features = self.activation(self.batchnorm(self.intermediate_linear(joint_features)))
        
        prediction = self.prediction_head(joint_features)
        
        return prediction

In [ ]:
model = SimpleModel().to(device=DEVICE).eval()

In [ ]:
def Reward(outputs, price):
    reward = outputs * price
    return reward

In [ ]:
data_X = torch.split(torch.load('data/proof_of_concept/X_test.pt').to(torch.float32), 1)
data_x = torch.split(torch.load('data/proof_of_concept/x_test.pt').to(torch.float32), 1)

In [ ]:
SH = data_X[0].shape

In [ ]:
M = len(data_X)
N = 1000
history = np.zeros((M, 3, N))

for i, (sample_X, sample_x) in enumerate(zip(data_X, data_x)):

    for j, price in enumerate(np.linspace(0, 500, N)):

        sample_x[0, -5] = price
        mean_b = model(sample_X.view(1, 11, -1).to(DEVICE), sample_x.to(DEVICE))
        reward = Reward(price, mean_b)

        history[i, 0, j] = price
        history[i, 1, j] = mean_b
        history[i, 2, j] = reward    

In [ ]:
fig, ax = plt.subplots(M, 2, figsize=(15, 10))

for i in range(M):
    ax[i, 0].plot(history[i, 0, :], history[i, 1, :])
    ax[i, 1].plot(history[i, 0, :], history[i, 2, :])
    
for ax_, col in zip(ax[0], ["Mean booked decades", "Reward"]):
    ax_.set_title(col)

for i in range(M):
    ylabel = ax[i, 0].set_ylabel(f"case {i+1}")
    ylabel.set_rotation(0)
    
    ax[i, 0].set_xlabel(f"Price")
    ax[i, 1].set_xlabel(f"Price")
    
plt.tight_layout()

In [ ]:
M = len(data_X)
N = 1000
history = np.zeros((M, 3, N))

for i, (sample_X, sample_x) in enumerate(zip(data_X, data_x)):

    for j, price in enumerate(np.linspace(0, 50, N)):

        sample_x[0, -5] = price
        mean_b = model(sample_X.view(1, 11, -1).to(DEVICE), sample_x.to(DEVICE))
        reward = Reward(price, mean_b)

        history[i, 0, j] = price
        history[i, 1, j] = mean_b
        history[i, 2, j] = reward    

In [ ]:
lbl = np.round(np.linspace(0, 50, 25), 0)

In [ ]:
fig, ax = plt.subplots(M, 2, figsize=(15, 10))
from matplotlib.ticker import MaxNLocator

for i in range(M):
    ax[i, 0].plot(history[i, 0, :], history[i, 1, :])
    ax[i, 1].plot(history[i, 0, :], history[i, 2, :])
    
for ax_, col in zip(ax[0], ["Mean booked decades", "Reward"]):
    ax_.set_title(col)

for i in range(M):
    ylabel = ax[i, 0].set_ylabel(f"case {i+1}")
    ylabel.set_rotation(0)
    
    ax[i, 0].set_xlabel(f"Price")
    ax[i, 1].set_xlabel(f"Price")
    
    ax[i, 0].set_xticks(lbl)
    ax[i, 0].set_xticklabels(lbl)
    ax[i, 1].set_xticks(lbl)
    ax[i, 1].set_xticklabels(lbl)
    
    ax[i, 0].tick_params(axis='x', labelsize=8)  
    ax[i, 1].tick_params(axis='x', labelsize=8)  
    


plt.tight_layout()